In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
import time
import numpy as np
import pandas as pd
import random
import re
import traceback

In [204]:
chrome_options = Options()
chrome_options.add_argument("--disabled-blink-features=AutomationControlled")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36")

driver_path = "C:/ds_study/driver/chromedriver-win64/chromedriver.exe"
service = Service(driver_path)
driver = webdriver.Chrome(service = service, options = chrome_options)
driver.implicitly_wait(5)
wait = WebDriverWait(driver, 5)
actions = ActionChains(driver)

driver.get("https://www.danawa.com/")
driver.maximize_window()



In [2]:

min_price_df = pd.DataFrame(columns=['product_name', 'mall', 'price', 'rank', 'major_category', 'middle_category', 'minor_category'])


In [ ]:
driver.close()

In [170]:
driver.quit()

In [161]:
windows = driver.window_handles
driver.switch_to.window(windows[-1])
driver.back()

In [153]:


try:
    # 첫 번째 카테고리 (메인 카테고리) 호버링
    base_main_category = '//*[@id="sectionLayer"]/li[2]/a'
    main_category_button = wait.until(EC.element_to_be_clickable((By.XPATH, base_main_category)))
    actions.move_to_element(main_category_button).perform()
    time.sleep(2)  # 요소가 로드되도록 약간의 대기

    # 두 번째 카테고리 (중간 카테고리) 호버링
    for i in range(1, 5):
        try:
            base_middle_category = '//*[@id="categoryHoverLayer10"]/div[1]/ul/li[{i}]/a/span'
            middle_category = base_middle_category.format(i = i)

            middle_category_button = wait.until(EC.visibility_of_element_located((By.XPATH, middle_category )))
            actions.move_to_element(middle_category_button).perform()
            time.sleep(2)  # 요소가 로드되도록 약간의 대기

            base_minor_category_button = '//*[@id="sectionLayer"]/li[2]//*[@id="categoryHoverLayer10"]\
                /div[1]/ul/li[2]//*[contains(@id, "categoryUnit")]//ul/li/a'  # 소분류 요소들
            minor_category_buttons = driver.find_elements(By.XPATH, base_minor_category_button)

            # 클릭 가능한 소분류 요소들을 수집
            clickable_elements = []
            for button in minor_category_buttons:
                if button.is_displayed() and button.is_enabled():
                    clickable_elements.append(button)

            # 수집된 요소들 출력
            print(f"Clickables: {len(clickable_elements)} elements found.")
            
            # 각 요소 클릭 (원하는 방식으로 사용)
            for index, element in enumerate(clickable_elements):
                print(f"Clicking element {index + 1}")
                driver.execute_script("arguments[0].click();", element)
                time.sleep(2)  # 페이지 로드 시간 대기

                # 필요하면 driver.back()으로 다시 돌아올 수 있습니다.
                driver.back()
                time.sleep(2)
        except:
            continue

except Exception as e:
    print(f"Error: {e}")


Clickables: 7 elements found.
Clicking element 1
Clicking element 2
Clicking element 3
Clicking element 4
Clicking element 5
Clicking element 6
Clicking element 7
Clickables: 0 elements found.


In [189]:
def collect_min_price():
    
    global min_price_df

    windows = driver.window_handles
    driver.switch_to.window(windows[-1])

    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '[id^="productItem"] > div > div.prod_info > p > a' )))
    products = driver.find_elements(By.CSS_SELECTOR, '[id^="productItem"] > div > div.prod_info > p > a' )


    for i4 in range(5):
        products[i4].click()
        time.sleep(2)
        
        windows = driver.window_handles
        driver.switch_to.window(windows[-1])
        
        
        base_xpath = '//*[@id="blog_content"]/div[2]/div[2]/div[1]/div[2]/div[3]/table/tbody[1]/tr/td[2]/a/span/em'
        base_xpath_mall = '//*[@id="blog_content"]/div[2]/div[2]/div[1]/div[2]/div[3]/table/tbody[1]/tr/td[1]/div/a/img'

        # price_list와 shopping_mall_list 초기화
        price_list = []
        shopping_mall_list = []

        # base_xpath로부터 해당하는 요소들이 몇 개인지 확인
        elements = driver.find_elements(By.XPATH, base_xpath)
        num_elements = len(elements)

        # 요소 개수만큼 루프
        for i in range(1, num_elements + 1):
            try:
                xpath = f'//*[@id="blog_content"]/div[2]/div[2]/div[1]/div[2]/div[3]/table/tbody[1]/tr[{i}]/td[2]/a/span/em'
                element = driver.find_element(By.XPATH, xpath)
                price_list.append(element.text)

                xpath_mall = f'//*[@id="blog_content"]/div[2]/div[2]/div[1]/div[2]/div[3]/table/tbody[1]/tr[{i}]/td[1]/div/a/img'
                mall_name = driver.find_element(By.XPATH, xpath_mall)
                alt_text = mall_name.get_attribute('alt')
                shopping_mall_list.append(alt_text)

            except:
                try:
                    base_xpath_no_img = f'//*[@id="blog_content"]/div[2]/div[2]/div[1]/div[2]/div[3]/table/tbody[1]/tr[{i}]/td[1]/div/a'
                    element = driver.find_element(By.XPATH, base_xpath_no_img)
                    company_name = element.text
                    shopping_mall_list.append(company_name)

                    continue

                except:
                    break
            print(f'{i} 번째 완료')

        # 가격 데이터를 정수형으로 변환
        price_list = [int(price.replace(',', '')) for price in price_list]
   

        df_price = pd.DataFrame( { 'mall' : shopping_mall_list, 'price' : price_list  })
        df_price['rank'] = df_price['price'].rank(method = 'min').astype(int)

        major_category_element = driver.find_element(By.XPATH, '//*[@id="sectionCategory"]/button')
        middle_category_element = driver.find_element(By.XPATH, '//*[@id="1DepthCategory"]/button')
        minor_category_element = driver.find_element(By.XPATH, '//*[@id="2DepthCategory"]/button')
        product_name_element = driver.find_element(By.XPATH, '//*[@id="blog_content"]/div[2]/div[1]/h3/span')


        major_category = major_category_element.text
        middle_category = middle_category_element.text
        minor_category = minor_category_element.text
        product_name = product_name_element.text

        df_price['major_category'] = major_category
        df_price['middle_category'] = middle_category
        df_price['minor_category'] = minor_category
        df_price['product_name'] = product_name
        min_price_df = pd.concat( [min_price_df, df_price], ignore_index= True  )

        driver.close()
        windows = driver.window_handles
        driver.switch_to.window(windows[0])
        time.sleep(2)

    print(min_price_df)

In [190]:
collect_min_price()

1 번째 완료
2 번째 완료
3 번째 완료
4 번째 완료
1 번째 완료
2 번째 완료
3 번째 완료
4 번째 완료
5 번째 완료
6 번째 완료
1 번째 완료
2 번째 완료
3 번째 완료
4 번째 완료
1 번째 완료
2 번째 완료
3 번째 완료
4 번째 완료
5 번째 완료
6 번째 완료
7 번째 완료
1 번째 완료
2 번째 완료
3 번째 완료
4 번째 완료
5 번째 완료
6 번째 완료
7 번째 완료
8 번째 완료
9 번째 완료
10 번째 완료
11 번째 완료
12 번째 완료
                     product_name       mall   price rank major_category  \
0        호카 본디 8 와이드 1127953-BBLC   HOKA 운동화  106000    1         골프/스포츠   
1        호카 본디 8 와이드 1127953-BBLC       11번가  116930    2         골프/스포츠   
2        호카 본디 8 와이드 1127953-BBLC         쿠팡  159000    3         골프/스포츠   
3        호카 본디 8 와이드 1127953-BBLC        G마켓  171640    4         골프/스포츠   
4        호카 본디 8 와이드 1127953-BBLC         옥션  176680    5         골프/스포츠   
..                            ...        ...     ...  ...            ...   
133  사조대림 해표 로스팅 꽃게 다시팩 81g (16개)    SSG.COM   96600    8       식품/유아/완구   
134  사조대림 해표 로스팅 꽃게 다시팩 81g (16개)  이마트인터넷쇼핑몰   96600    8       식품/유아/완구   
135  사조대림 해표 로스팅 꽃게 다시팩 81g (16개)       신세계몰   96

In [313]:
def gather_price_data():
    
    chrome_options = Options()
    chrome_options.add_argument("--disabled-blink-features=AutomationControlled")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36")
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920X1080")


    driver_path = "C:/ds_study/driver/chromedriver-win64/chromedriver.exe"
    service = Service(driver_path)
    driver = webdriver.Chrome(service = service, options = chrome_options)
    driver.implicitly_wait(5)
    wait = WebDriverWait(driver, 5)
    actions = ActionChains(driver)

    driver.get("https://www.danawa.com/")
    time.sleep(2)

    try:
        i2 = 0
     
        for i in range(4, 6): # (2,13)이 전체
            base_main_category = '//*[@id="sectionLayer"]/li[{i}]/a'
            main_category = base_main_category.format(i = i)
            main_category_button = driver.find_element(By.XPATH, main_category)


            for i2 in range(1, 10): # (1,20) 정도가 적당
                base_middle_category = '//*[@id="categoryHoverLayer{i1}"]/div[1]/ul/li[{i2}]/a/span'
                actions.move_to_element(main_category_button).perform()
                try:
                    middle_category = base_middle_category.format(i1 = i + 8, i2 = i2)
                    middle_category_button = driver.find_element(By.XPATH, middle_category)
                    
                    if middle_category_button:
                        actions.move_to_element(middle_category_button).perform()
                        time.sleep(2) 
                
                        base_minor_category_button = (
                            '//*[@id="sectionLayer"]/li[{a}]\
                            //*[@id="categoryHoverLayer{b}"]/div[1]/ul/li[{c}]\
                            //*[contains(@id, "categoryUnit")]/ul/li/a'
                            )
                        
                        
                        minor_category_buttons = driver.find_elements(By.XPATH, base_minor_category_button.format(a = i, \
                            b = i + 8, c = i2))
                        
                        if len(minor_category_buttons) > 0:
                            clickable_elements = []
                            for button in minor_category_buttons:
                                if button.is_displayed() and button.is_enabled():
                                    clickable_elements.append(button)

                            for each in clickable_elements:
                                each.click()
                                time.sleep(2)
                                
                                collect_min_price()


                                driver.back()
                                windows = driver.window_handles
                                driver.switch_to.window(windows[0])
                                time.sleep(1)
                                
                                main_category_button = wait.until(EC.element_to_be_clickable((By.XPATH, main_category)))
                                actions.move_to_element(main_category_button).perform()
                                middle_category_button = wait.until(EC.element_to_be_clickable((By.XPATH, middle_category)))
                                actions.move_to_element(middle_category_button).perform()

                        else:
                            print(f"{i2}번째 중분류에서 소분류가 나타나지 않음")
                    
                            
                except Exception as e:
                    print(f'{e} : {i-1}번째 대분류, {i2}번째에서 중분류 발생')
                    continue
                                

    except Exception as e:
        print(f'{e} : {i-1}번째 대분류, {i2}번째 중분류에서 오류 발생')
    driver.quit()
    
print(min_price_df.head(10))
print(len(min_price_df))


               product_name      mall   price rank major_category  \
0  호카 본디 8 와이드 1127953-BBLC  HOKA 운동화  106000    1         골프/스포츠   
1  호카 본디 8 와이드 1127953-BBLC      11번가  116930    2         골프/스포츠   
2  호카 본디 8 와이드 1127953-BBLC        쿠팡  159000    3         골프/스포츠   
3  호카 본디 8 와이드 1127953-BBLC       G마켓  171640    4         골프/스포츠   
4  호카 본디 8 와이드 1127953-BBLC        옥션  176680    5         골프/스포츠   
5  호카 본디 8 와이드 1127953-BBLC   현대Hmall  191040    6         골프/스포츠   
6  호카 본디 8 와이드 1127953-BBLC      롯데ON  193840    7         골프/스포츠   
7  호카 본디 8 와이드 1127953-BBLC       GS샵  199210    8         골프/스포츠   
8  호카 본디 8 와이드 1127953-BBLC   SSG.COM  199805    9         골프/스포츠   
9  호카 본디 8 와이드 1127953-BBLC      신세계몰  200321   10         골프/스포츠   

  middle_category minor_category  
0            스포츠화      스포츠화 전체보기  
1            스포츠화      스포츠화 전체보기  
2            스포츠화      스포츠화 전체보기  
3            스포츠화      스포츠화 전체보기  
4            스포츠화      스포츠화 전체보기  
5            스포츠화      스포츠화 전체

In [314]:
gather_price_data()

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="categoryHoverLayer12"]/div[1]/ul/li[1]/a/span"}
  (Session info: chrome=129.0.6668.100); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6933EFDF2+29714]
	(No symbol) [0x00007FF693368069]
	(No symbol) [0x00007FF69322B9DA]
	(No symbol) [0x00007FF69327F657]
	(No symbol) [0x00007FF69327F8AC]
	(No symbol) [0x00007FF6932C9CA7]
	(No symbol) [0x00007FF6932A6C7F]
	(No symbol) [0x00007FF6932C6A82]
	(No symbol) [0x00007FF6932A69E3]
	(No symbol) [0x00007FF69327114A]
	(No symbol) [0x00007FF6932722B1]
	GetHandleVerifier [0x00007FF69371020D+3307565]
	GetHandleVerifier [0x00007FF69375CCE3+3621635]
	GetHandleVerifier [0x00007FF693752087+3577511]
	GetHandleVerifier [0x00007FF6934A7456+780918]
	(No symbol) [0x00007FF6933731AF]
	(No symbol) [0x00007FF69336ED34]
	(No symbol) [0x00

In [193]:
print(len(min_price_df))

402


In [146]:
gather_price_data()


Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="categoryHoverLayer10"]/div[1]/ul/li[1]/a/span"}
  (Session info: chrome=129.0.6668.100); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6933EFDF2+29714]
	(No symbol) [0x00007FF693368069]
	(No symbol) [0x00007FF69322B9DA]
	(No symbol) [0x00007FF69327F657]
	(No symbol) [0x00007FF69327F8AC]
	(No symbol) [0x00007FF6932C9CA7]
	(No symbol) [0x00007FF6932A6C7F]
	(No symbol) [0x00007FF6932C6A82]
	(No symbol) [0x00007FF6932A69E3]
	(No symbol) [0x00007FF69327114A]
	(No symbol) [0x00007FF6932722B1]
	GetHandleVerifier [0x00007FF69371020D+3307565]
	GetHandleVerifier [0x00007FF69375CCE3+3621635]
	GetHandleVerifier [0x00007FF693752087+3577511]
	GetHandleVerifier [0x00007FF6934A7456+780918]
	(No symbol) [0x00007FF6933731AF]
	(No symbol) [0x00007FF69336ED34]
	(No symbol) [0x00

In [354]:
min_price_df.to_csv('최저가_데이터2.csv', index = False, encoding = 'utf-8-sig')

In [195]:
review_collective_df = pd.DataFrame(columns = ['title', 'review', 'score', 'mall', 'date' ,'reviewer'] )

In [315]:
chrome_options = Options()
chrome_options.add_argument("--disabled-blink-features=AutomationControlled")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36")

driver_path = "C:/ds_study/driver/chromedriver-win64/chromedriver.exe"
service = Service(driver_path)
driver = webdriver.Chrome(service = service, options = chrome_options)
driver.implicitly_wait(5)
wait = WebDriverWait(driver, 5)
actions = ActionChains(driver)

driver.get('https://prod.danawa.com/list/?cate=16228134&15main_16_02=')
driver.maximize_window()


In [316]:

windows = driver.window_handles
driver.switch_to.window(windows[0])
# a 태그들 중에서 텍스트가 "상품평 많은순"인 요소를 찾아 클릭

sort_by_review = driver.find_element(By.XPATH, '//a[@href = "#" and text()="상품평 많은순"]')
sort_by_review.click()


In [349]:
def collect_review():
    global review_collective_df

    windows = driver.window_handles
    driver.switch_to.window(windows[0])
    
    sort_by_review = driver.find_element(By.XPATH, '//a[@href = "#" and text()="상품평 많은순"]')
    sort_by_review.click()

    time.sleep(2)

    try:
        for i4 in range(5):
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '[id^="productItem"] > div > div.prod_info > p > a' )))
            products = driver.find_elements(By.CSS_SELECTOR, '[id^="productItem"] > div > div.prod_info > p > a' )
            products[i4].click()
            time.sleep(2)
            
            windows = driver.window_handles
            driver.switch_to.window(windows[-1])
            review_button = driver.find_element(By.XPATH, '//h4[@class="txt" and text()="쇼핑몰 상품리뷰"]')
            review_button.click()
            time.sleep(2)

            title_list = []
            review_list = []
            score_list = []
            mall_list = []
            date_list = []
            reviewer_list = []

            title_base = '//*[@id="danawa-prodBlog-companyReview-content-wrap-{i}"]/div[1]/div[1]/p'
            review_base = '//*[@id="danawa-prodBlog-companyReview-content-wrap-{i}"]/div/div[2]'

            score_base = '//*[@id="danawa-prodBlog-companyReview-content-list"]/ul/li[{i}]/div[1]'
            mall_base = '//*[@id="danawa-prodBlog-companyReview-content-list"]/ul/li[{i}]/div[1]/span[3]/img'
            date_base = '//*[@id="danawa-prodBlog-companyReview-content-list"]/ul/li[{i}]/div[1]/span[5]'
            reviewer_base = '//*[@id="danawa-prodBlog-companyReview-content-list"]/ul/li[{i}]/div[1]/span[7]'


            for i in range(12):
                try:
                    title_xpath = title_base.format(i = i)
                    title_element = driver.find_element(By.XPATH, title_xpath)
                    title_list.append(title_element.text)
                
                    review_xpath = review_base.format(i = i)
                    review_element = driver.find_element(By.XPATH, review_xpath)
                    review_list.append(review_element.text)

                    score_xpath = score_base.format(i = i + 1)
                    score_element = driver.find_element(By.XPATH, score_xpath)
                    pattern = r'\d+점'
                    result = re.findall(pattern, score_element.text)
                    score_list.extend(result)

                    mall_xpath = mall_base.format(i = i + 1)
                    mall_element = driver.find_element(By.XPATH, mall_xpath)
                    mall_list.append(mall_element.get_attribute('alt'))


                    date_xpath = date_base.format(i = i + 1)
                    date_element = driver.find_element(By.XPATH, date_xpath)
                    date_list.append(date_element.text)
                
                    
                    reviewer_xpath = reviewer_base.format(i = i + 1)
                    reviewer_element = driver.find_element(By.XPATH, reviewer_xpath)
                    reviewer_list.append(reviewer_element.text)



                except:
                    print(f'{i} 개 리뷰')
                    break

            review_df = pd.DataFrame({ 'title': title_list, 'review' : review_list, 'score' : score_list,  'mall' : mall_list, 'date' : date_list,\
                                    'reviewer' : reviewer_list })
            review_collective_df = pd.concat([review_collective_df, review_df], ignore_index= True)
            try:
                for page_num in range(2, 12):
                # 해당 페이지 번호의 버튼을 찾는 XPath
                    page_button_xpath = f'//a[@data-pagenumber="{page_num}"]'

                    # 페이지 버튼이 클릭 가능할 때까지 기다림
                    page_button = driver.find_element(By.XPATH, page_button_xpath)
                    
                    # 페이지 버튼 클릭
                    page_button.click()
                    
                    # 페이지가 로드될 시간을 주기 위해 대기 (페이지 로드 시간에 맞춰 조정 가능)
                    time.sleep(2)
                    title_list = []
                    review_list = []
                    score_list = []
                    mall_list = []
                    date_list = []
                    reviewer_list = []

                    title_base = '//*[@id="danawa-prodBlog-companyReview-content-wrap-{i}"]/div[1]/div[1]/p'
                    review_base = '//*[@id="danawa-prodBlog-companyReview-content-wrap-{i}"]/div/div[2]'

                    score_base = '//*[@id="danawa-prodBlog-companyReview-content-list"]/ul/li[{i}]/div[1]'
                    mall_base = '//*[@id="danawa-prodBlog-companyReview-content-list"]/ul/li[{i}]/div[1]/span[3]/img'
                    date_base = '//*[@id="danawa-prodBlog-companyReview-content-list"]/ul/li[{i}]/div[1]/span[5]'
                    reviewer_base = '//*[@id="danawa-prodBlog-companyReview-content-list"]/ul/li[{i}]/div[1]/span[7]'


                    for i in range(11):
                        try:
                            title_xpath = title_base.format(i = i)
                            title_element = driver.find_element(By.XPATH, title_xpath)
                            title_list.append(title_element.text)
                        
                            review_xpath = review_base.format(i = i)
                            review_element = driver.find_element(By.XPATH, review_xpath)
                            review_list.append(review_element.text)

                            score_xpath = score_base.format(i = i + 1)
                            score_element = driver.find_element(By.XPATH, score_xpath)
                            pattern = r'\d+점'
                            result = re.findall(pattern, score_element.text)
                            score_list.extend(result)

                            mall_xpath = mall_base.format(i = i + 1)
                            mall_element = driver.find_element(By.XPATH, mall_xpath)
                            mall_list.append(mall_element.get_attribute('alt'))


                            date_xpath = date_base.format(i = i + 1)
                            date_element = driver.find_element(By.XPATH, date_xpath)
                            date_list.append(date_element.text)
                        
                            
                            reviewer_xpath = reviewer_base.format(i = i + 1)
                            reviewer_element = driver.find_element(By.XPATH, reviewer_xpath)
                            reviewer_list.append(reviewer_element.text)



                        except:
                            print(f'{i} 개 리뷰')
                            break

                    review_df = pd.DataFrame({ 'title': title_list, 'review' : review_list, 'score' : score_list,  'mall' : mall_list, 'date' : date_list,\
                                            'reviewer' : reviewer_list })
                    review_collective_df = pd.concat([review_collective_df, review_df], ignore_index= True)
            except:
                continue

            driver.close()
            windows = driver.window_handles
            driver.switch_to.window(windows[0])
            time.sleep(2)
    except:
        print('에러발생')
        

In [318]:
collect_review()

10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰
10 개 리뷰


In [350]:
def gather_review_data():
    
    chrome_options = Options()
    chrome_options.add_argument("--disabled-blink-features=AutomationControlled")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36")
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920X1080")


    driver_path = "C:/ds_study/driver/chromedriver-win64/chromedriver.exe"
    service = Service(driver_path)
    driver = webdriver.Chrome(service = service, options = chrome_options)
    driver.implicitly_wait(5)
    wait = WebDriverWait(driver, 5)
    actions = ActionChains(driver)

    driver.get("https://www.danawa.com/")
    time.sleep(2)
    
    i2 = 0
    
    for i in range(2, 4): # (2,13)이 전체
        try:
            base_main_category = '//*[@id="sectionLayer"]/li[{i}]/a'
            main_category = base_main_category.format(i = i)
            main_category_button = driver.find_element(By.XPATH, main_category)


            for i2 in range(1, 5): # (1,20) 정도가 적당
                base_middle_category = '//*[@id="categoryHoverLayer{i1}"]/div[1]/ul/li[{i2}]/a/span'
                actions.move_to_element(main_category_button).perform()
                try:
                    middle_category = base_middle_category.format(i1 = i + 8, i2 = i2)
                    middle_category_button = driver.find_element(By.XPATH, middle_category)
                    
                    if middle_category_button:
                        actions.move_to_element(middle_category_button).perform()
                        time.sleep(2) 
                
                        base_minor_category_button = (
                            '//*[@id="sectionLayer"]/li[{a}]\
                            //*[@id="categoryHoverLayer{b}"]/div[1]/ul/li[{c}]\
                            //*[contains(@id, "categoryUnit")]/ul/li/a'
                            )
                        
                        
                        minor_category_buttons = driver.find_elements(By.XPATH, base_minor_category_button.format(a = i, \
                            b = i + 8, c = i2))
                        
                        if len(minor_category_buttons) > 0:
                            clickable_elements = []
                            for button in minor_category_buttons:
                                if button.is_displayed() and button.is_enabled():
                                    clickable_elements.append(button)
                            for each in clickable_elements:
                                each.click()
                            #for index, element in enumerate(clickable_elements):            
                                # driver.execute_script("arguments[0].click();", element)
                                time.sleep(2)
                                print('collect_review()실행 직전')
                                collect_review()


                                driver.get("https://www.danawa.com/")
                                windows = driver.window_handles
                                driver.switch_to.window(windows[-1])
                                time.sleep(2)
                                
                                main_category_button = wait.until(EC.element_to_be_clickable((By.XPATH, main_category)))
                                actions.move_to_element(main_category_button).perform()
                                middle_category_button = wait.until(EC.element_to_be_clickable((By.XPATH, middle_category)))
                                actions.move_to_element(middle_category_button).perform()

                        else:
                            print(f"{i2}번째 중분류에서 소분류가 나타나지 않음")
                    
                            
                except Exception as e:
                    print(f'{e} : {i-1}번째 대분류, {i2}번째에서 중분류 발생')
                    continue
                            

        except Exception as e:
            print(f'{e} : {i-1}번째 대분류, {i2}번째 중분류에서 오류 발생')
    driver.quit()
    

In [351]:
gather_review_data()

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="categoryHoverLayer10"]/div[1]/ul/li[1]/a/span"}
  (Session info: chrome=129.0.6668.100); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6933EFDF2+29714]
	(No symbol) [0x00007FF693368069]
	(No symbol) [0x00007FF69322B9DA]
	(No symbol) [0x00007FF69327F657]
	(No symbol) [0x00007FF69327F8AC]
	(No symbol) [0x00007FF6932C9CA7]
	(No symbol) [0x00007FF6932A6C7F]
	(No symbol) [0x00007FF6932C6A82]
	(No symbol) [0x00007FF6932A69E3]
	(No symbol) [0x00007FF69327114A]
	(No symbol) [0x00007FF6932722B1]
	GetHandleVerifier [0x00007FF69371020D+3307565]
	GetHandleVerifier [0x00007FF69375CCE3+3621635]
	GetHandleVerifier [0x00007FF693752087+3577511]
	GetHandleVerifier [0x00007FF6934A7456+780918]
	(No symbol) [0x00007FF6933731AF]
	(No symbol) [0x00007FF69336ED34]
	(No symbol) [0x00

In [352]:
review_collective_df.to_csv('리뷰데이터.csv', index = False, encoding = 'utf-8-sig')

In [22]:
driver.close()

In [312]:
driver.quit()

In [25]:
driver.back()